<a href="https://colab.research.google.com/github/vmatiasw/modelos_y_simulacion/blob/main/P4E10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
from collections import Counter
import random
import time
import math

def medir_tiempo(func):
    inicio = time.perf_counter()
    resultado = func()
    duracion = time.perf_counter() - inicio
    return resultado, duracion

def obtener_fpm_dict(df_fpm):
    df_fpm_fpm = df_fpm[df_fpm['Tema'] == 'FPM']
    return dict(zip(df_fpm_fpm['Datos'], df_fpm_fpm['Valor']))

def calcular_TVD(dict_d1, dict_d2): # Distancia Total de Variación.
    soporte_total = set(dict_d1) | set(dict_d2)
    return 0.5 * sum(abs(dict_d1.get(k, 0) - dict_d2.get(k, 0)) for k in soporte_total)

# --------------------------- Pandas ---------------------------
def comparar_dfs(df_A, df_B):
    df_comparado = pd.merge(df_A[['Tema', 'Datos', 'Valor']],
                            df_B[['Datos', 'Valor']],
                            on='Datos',
                            how='outer',
                            suffixes=('_A', '_B'))

    df_comparado['Diferencia'] = (df_comparado['Valor_A'] - df_comparado['Valor_B']).abs()
    return df_comparado[['Tema', 'Datos', 'Valor_A', 'Valor_B', 'Diferencia']]

def crear_df_fpm(dict_fpm):
    soporte = list(dict_fpm.keys())
    data = [{
        'Tema': 'FPM',
        "Datos": x,
        "Valor": dict_fpm.get(x, 0)
    } for x in soporte]

    esp = sum(x * p for x, p in dict_fpm.items())
    var = sum((x - esp)**2 * p for x, p in dict_fpm.items())
    desv = var**0.5
    resumen = [
        {'Tema': 'Estadisticas', "Datos": "Esperanza", "Valor": esp},
        {'Tema': 'Estadisticas', "Datos": "Varianza", "Valor": var},
        {'Tema': 'Estadisticas', "Datos": "Desv. Est.", "Valor": desv},
    ]

    return pd.concat([pd.DataFrame(data), pd.DataFrame(resumen)], ignore_index=True)

def estimar_df_fpm(generador_de_muestras, n=100_000):
    muestras, tiempos = zip(*(medir_tiempo(generador_de_muestras) for _ in range(n)))

    frec = Counter(muestras)
    soporte = sorted(list(frec.keys()))

    dict_fpm_estimada = {x: frec.get(x, 0) / n for x in soporte}
    df_fpm_estimada = crear_df_fpm(dict_fpm_estimada)

    tiempo_promedio = sum(tiempos) / len(tiempos)
    df_tiempo_promedio = pd.DataFrame([{
        'Tema': 'Performance',
        "Datos": "Tiempo Promedio",
        "Valor": tiempo_promedio
    }])

    return pd.concat([df_fpm_estimada, df_tiempo_promedio], ignore_index=True)

# --------------------------- Utilidades ---------------------------

def ordenar_dict_fpm_por_probabilidad(dict_fpm):
  return dict(sorted(dict_fpm.items(), key=lambda x: x[1], reverse=True))

def calcular_dict_fda(dict_fpm):
  dict_fda = {}
  acc = 0
  for v, p in dict_fpm.items():
    acc += p
    dict_fda[v] = acc
  return dict_fda

# def crear_dict_fpm(soporte, fpm):
#   return {v: fpm(v) for v in soporte}

# dict_A = dict(gen_A)
# gen_A = iter(dict_A.items())

def probabilidad(condicion, n):
    exitos = sum(condicion() for _ in range(n))
    return exitos / n

def gen_fpm_truncada(gen_fpm, condicion):
  list_fpm = list(gen_fpm())
  suma_probabilidades = sum(p for i, p in list_fpm if condicion(i))

  for i, p in list_fpm:
    if condicion(i):
      yield i, p / suma_probabilidades

def gen_fpm(gen_soporte, fpm):
  soporte = gen_soporte()
  fda_v = 0
  while fda_v < 1 - 1e-15:
    v = next(soporte)
    fpm_v = fpm(v)
    yield v, fpm_v
    fda_v += fpm_v

def gen_fda(gen_soporte, fpm):
  soporte = gen_soporte()
  fda_v = 0
  while fda_v < 1 - 1e-15:
    v = next(soporte)
    fda_v += fpm(v)
    yield v, fda_v

def TI(gen_soporte, fpm):
  u = random.random()
  soporte = gen_soporte()
  fda_v = 0
  for v in soporte:
    fda_v += fpm(v)
    if u <= fda_v:
      return v

# dict_A = dict(gen_A)
# def TI(dict_fda):
#   u = random.random()
#   for v, p in dict_fda.items():
#     if u < p:
#       return v

def AyR(dict_fpm_objetivo, fpm_propuesta, dict_fpm_propuesta, C):
    while True:
        muestra = fpm_propuesta()
        prob_objetivo = dict_fpm_objetivo.get(muestra, 0)
        prob_propuesta = dict_fpm_propuesta.get(muestra, 0)

        if prob_propuesta == 0:
            continue

        if random.random() < prob_objetivo / (C * prob_propuesta):
            return muestra

In [40]:
# Ejercicio 10.
# (a) Desarrolle un método para generar una variable aleatoria X cuya
# distribución de probabilidad está dada por: ...

def fpm(v): # v = 1, 2, ...
  return (1 / 2**(v+1)) + (2**(v-2) / 3**v)

def gen_soporte():
    conteo = 1
    while True:
        yield conteo
        conteo += 1

dict_fpm = dict(gen_fpm(gen_soporte, fpm))
df_fpm = crear_df_fpm(dict_fpm)

generador_de_muestras = lambda: TI(gen_soporte, fpm)
df_fpm_estimada = estimar_df_fpm(generador_de_muestras)
dict_fpm_estimada = obtener_fpm_dict(df_fpm_estimada)

print(calcular_TVD(dict_fpm, dict_fpm_estimada))
comparar_dfs(df_fpm, df_fpm_estimada)

0.004580566160737524


,Tema,Datos,Valor_A,Valor_B,Diferencia
0,FPM,1,4.166667e-01,0.417360,0.000693
1,FPM,2,2.361111e-01,0.232700,0.003411
2,FPM,3,1.365741e-01,0.137530,0.000956
3,FPM,4,8.063272e-02,0.082140,0.001507
4,FPM,5,4.854681e-02,0.047490,0.001057
...,...,...,...,...,...
83,FPM,84,4.039004e-16,NaN,NaN
84,Estadisticas,Desv. Est.,2.061553e+00,2.078174,0.016621
85,Estadisticas,Esperanza,2.500000e+00,2.509600,0.009600
86,NaN,Tiempo Promedio,NaN,0.000002,NaN
